# Classification 02

# Resampling and evaluation of prediction methods

#### Josep Fortiana 2019-10-29

The goal of this laboratory is to attain a practical knowledge of some of the available procedures to assess the quality of prediction methods, classification (categorical response) and regression (numerical response), with a particular emphasis on detecting and avoiding overfitting.

We consider four procedures or families of procedures: 
    
1. Hold-out: Split the dataset in two parts, a training subset and a 'test' subset. The first one is used to train (estimate or learn parameters of) the prediction method. Meanwhile, the 'test' subset is held out, keeping it apart to rule out the optimistic bias inherent in using the same samples for learning to predict and for evaluating goodness of prediction.

2. $k$-fold cross-validation,  

3. Leave-one-out (LOO), 

4. Bootstrap.

We already know the first procedure. The second one is a systematic repetition of hold-out in such a way that consecutively each individual is in a training subset and in a test subset. The third procedure is an extreme $k$-fold cross-validation, where $k=n$, the number of individuals in the sample, in which the test subset is a single individual and the training subset  the remaining $(n-1)$ restantes. 

The fourth procedure is an application of a much more general statistical concept, the _bootstrap,_ which we review in Section A, before going to its application with the other procedures.

# Datasets and prediction methods

# A. `wine` dataset

`wine` data are the results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines.

They can be found in the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/), which hosts many documented data sets to be used as benchmarks in evaluating Machine Learning methods and algorithms. Alternatively, should the link be broken, you can find the `.csv` file in the Virtual Campus. The following description is taken from the UCI website:

I think that the initial data set had around 30 variables, but for some reason I only have the 13 dimensional version. I had a list of what the 30 or so variables were, but a.) I lost it, and b.), I would not know which 13 variables are included in the set.

The attributes are:

01. Alcohol

02. Malic acid

03. Ash

04. Alcalinity of ash

05. Magnesium

06. Total phenols

07. Flavonoids

08. Nonflavonoid phenols

09. Proanthocyanins

10. Color intensity

11. Hue

12. OD280/OD315 of diluted wines

13. Proline


In a classification context, this is a well posed problem with "well behaved" class structures. A good data set for first testing of a new classifier, but not very challenging.

Since the `.csv` file has no first row with variable names we must set `header=FALSE` in the `read.csv` call (see default values for the optional parameters in the help). 

The casting `as.factor()` command has the purpose of conveying the fact that this variable is qualitative, so the R interpreter can use it as such.


In [105]:
wine.url<-"http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
#wine<-read.csv(wine.url,header=FALSE)
wine<-read.csv("wine.csv",header=FALSE)
colnames(wine)<-c("Type","Alcohol","Malic","Ash", "Alcalinity","Magnesium","Phenols","Flavonoids", 
                  "Nonflavonoids","Proanthocyanins","Color","Hue", "Dilution","Proline")
wine$Type <- as.factor(wine$Type)

## Aa. `wine` dataset and $k$-NN

### Aa1. Hold-out

Split the whole dataset into two subsets, `train` with $\approx60\%$ of data, and `test` the rest.

In [106]:
summary(wine)
n<-nrow(wine)
ntrain<-ceiling(0.6*n)
ntest<-n-ntrain
Itrain<-sample(1:n,ntrain,replace=FALSE)
wine.train<-wine[Itrain,]
wine.test<-wine[-Itrain,]
Xtrain<-as.matrix(wine.train[,-1])
ytrain<-wine.train[,1]
Xtest<-as.matrix(wine.test[,-1])
ytest<-wine.test[,1]

 Type      Alcohol          Malic            Ash          Alcalinity   
 1:59   Min.   :11.03   Min.   :0.740   Min.   :1.360   Min.   :10.60  
 2:71   1st Qu.:12.36   1st Qu.:1.603   1st Qu.:2.210   1st Qu.:17.20  
 3:48   Median :13.05   Median :1.865   Median :2.360   Median :19.50  
        Mean   :13.00   Mean   :2.336   Mean   :2.367   Mean   :19.49  
        3rd Qu.:13.68   3rd Qu.:3.083   3rd Qu.:2.558   3rd Qu.:21.50  
        Max.   :14.83   Max.   :5.800   Max.   :3.230   Max.   :30.00  
   Magnesium         Phenols        Flavonoids    Nonflavonoids   
 Min.   : 70.00   Min.   :0.980   Min.   :0.340   Min.   :0.1300  
 1st Qu.: 88.00   1st Qu.:1.742   1st Qu.:1.205   1st Qu.:0.2700  
 Median : 98.00   Median :2.355   Median :2.135   Median :0.3400  
 Mean   : 99.74   Mean   :2.295   Mean   :2.029   Mean   :0.3619  
 3rd Qu.:107.00   3rd Qu.:2.800   3rd Qu.:2.875   3rd Qu.:0.4375  
 Max.   :162.00   Max.   :3.880   Max.   :5.080   Max.   :0.6600  
 Proanthocyanins     Color 

In [107]:
#install.packages("class",dependencies=TRUE,repos="https://cloud.r-project.org")
require(class)

#### Confusion matrix and misclassification error estimate

In [108]:
require(ISLR)
require(caret)

set.seed(400)
ctrl <- trainControl(method="repeatedcv",repeats = 3)
knnFit <- train(ytrain, data = Xtrain, method = "knn", trControl = ctrl, preProcess = c("center","scale"),tuneLength = 20)
knnFit

k<-3
y.hat<-knn(Xtrain,Xtest,ytrain,k )
C<-table("True"=ytest,"Predicted"=y.hat)
P.err<-(ntest-sum(diag(C)))/ntest
C
sprintf("For k =%d,   Prob.err = %5.2f ", k, round(P.err,2))

    Predicted
True  1  2  3
   1 21  3  2
   2  2 18  5
   3  3  7 10

[1] "For k =3,   Prob.err =  0.31 "

#### Which is the optimal $k$?

Redo the above computation with several values of $k$ and decide which is the optimal $k$ according to the probability of misclassification.

_Hint:_ After doing it _"by hand"_ you can try a systematic approach, building a table where the first column is `k` and the second column the estimated probability of misclassification.

### Aa.2. k-fold cross-validation

Here we perform $k$-fold cross-validation with some simple code. An alternative approach is to use functions from a dedicated R package. My personal prejudice is in favor of writing one's own code. Anyway you can see in the Appendix at the end of this notebook some possible such packages and you can judge by yourselves.

In [164]:
kfoldIndexes<-function(n,k){
    l<-floor(n/k)
    Indexes<-c(1,(1:k)*l)
    Indexes[k+1]<-n
    return(Indexes)
    }

In [167]:
# Decide a 'K' for K-fold cross-validation (uppercase to avoid notation clash with the k in k-NN)
n<-nrow(wine)
K<-3
J<-kfoldIndexes(n,K)
J
Lower<-J[-(K+1)]
Upper<-J[-1]
Lower
Upper
# Una random permutation of indexes
I<-sample(1:n)

[1]   1  59 118 178

[1]   1  59 118

[1]  59 118 178

In [168]:
k<-10# this is the 'k' in k-NN
# Repeat with several 'k' and choose an optimal value.
P.ERR<-rep(0,K)
for (fold in 1:K){
    Itest<-I[Lower[fold]:Upper[fold]]
    wine.test<-wine[Itest,]
    wine.train<-wine[-Itest,]
    Xtrain<-as.matrix(wine.train[,-1])
    ytrain<-wine.train[,1]
    Xtest<-as.matrix(wine.test[,-1])
    ytest<-wine.test[,1]
    y.hat<-knn(Xtrain,Xtest,ytrain,k)
    C<-table("True"=ytest,"Predicted"=y.hat)
    print(C)
    print(fold)
    P.ERR[fold]<-(ntest-sum(diag(C)))/ntest
    print(P.ERR[fold])
    }
round(P.ERR,3)
mean.p.err<-mean(P.ERR)
round(mean.p.err,3)

    Predicted
True  1  2  3
   1 19  0  1
   2  2 16  7
   3  2  7  5
[1] 1
[1] 0.4366197
    Predicted
True  1  2  3
   1 15  1  2
   2  0 17  6
   3  2  5 12
[1] 2
[1] 0.3802817
    Predicted
True  1  2  3
   1 20  0  2
   2  2 13  9
   3  0  3 12
[1] 3
[1] 0.3661972


[1] 0.437 0.380 0.366

[1] 0.394

### Aa.3. Leave-one-out _(LOO)_

In [171]:
k<-6
# Repeat with several 'k' and choose an optimal value.
g<-nlevels(wine$Type)
C<-matrix(0,nrow=g,ncol=g)
for (i in 1:n){
    wine.test<-wine[i,]
    wine.train<-wine[-i,]
    Xtrain<-as.matrix(wine.train[,-1])
    ytrain<-wine.train[,1]
    Xtest<-as.matrix(wine.test[,-1])
    ytest<-wine.test[,1]
    y.hat<-knn(Xtrain,Xtest,ytrain,k)
    C[ytest,y.hat]=C[ytest,y.hat]+1
    }
print(C)
p.err<-(n-sum(diag(C)))/n
round(p.err,3)

     [,1] [,2] [,3]
[1,]   52    1    6
[2,]    5   47   19
[3,]    5   20   23


[1] 0.315

### Aa.4.  _bootstrap_

In [24]:
k<-7
# Repeat with several 'k' and choose an optimal value.
n<-nrow(wine)
I<-1:n
# Number of bootstrap resamples
B<-10
P.ERR<-rep(0,B)
for (b in 1:B){
    Ib<-sample(I,n,replace = TRUE)
    oob<-I[is.na(match(I,Ib))]
    Itest<-oob
    ntest<-length(oob)
    print(ntest)
    wine.test<-wine[Itest,]
    wine.train<-wine[-Itest,]
    Xtrain<-as.matrix(wine.train[,-1])
    ytrain<-wine.train[,1]
    Xtest<-as.matrix(wine.test[,-1])
    ytest<-wine.test[,1]
    y.hat<-knn(Xtrain,Xtest,ytrain,k)
    C<-table("True"=ytest,"Predicted"=y.hat)
    print(C)
    P.ERR[b]<-(ntest-sum(diag(C)))/ntest
    }
round(P.ERR,3)
mean.p.err<-mean(P.ERR)
round(mean.p.err,3)

[1] 67
    Predicted
True  1  2  3
   1 17  0  2
   2  4 19  5
   3  2 11  7
[1] 73
    Predicted
True  1  2  3
   1 22  0  0
   2  3 18  8
   3  3  8 11
[1] 75
    Predicted
True  1  2  3
   1 22  0  6
   2  2 19 10
   3  1  4 11
[1] 66
    Predicted
True  1  2  3
   1 17  1  4
   2  1 20  5
   3  0  8 10
[1] 62
    Predicted
True  1  2  3
   1 20  0  2
   2  1 16 12
   3  1  1  9
[1] 62
    Predicted
True  1  2  3
   1 19  0  2
   2  1 18  3
   3  4  6  9
[1] 56
    Predicted
True  1  2  3
   1 16  0  2
   2  0 15  7
   3  3  5  8
[1] 71
    Predicted
True  1  2  3
   1 21  0  2
   2  2 18  9
   3  3  8  8
[1] 65
    Predicted
True  1  2  3
   1 17  1  1
   2  3 18  4
   3  1 13  7
[1] 63
    Predicted
True  1  2  3
   1 20  0  1
   2  1  9 14
   3  1  3 14


[1] 0.358 0.301 0.307 0.288 0.274 0.258 0.304 0.338 0.354 0.317

[1] 0.31

# B. `Auto` dataset

#### Description
Gas mileage, horsepower, and other information for 392 vehicles.

#### Format
A data frame with 392 observations on the following 9 variables.

01. `mpg`: miles per gallon.

02. `cylinders`: Number of cylinders between 4 and 8.

03. `displacement`: Engine displacement (cu. inches).

04. `horsepower`: Engine horsepower.

05. `weight`: Vehicle weight (lbs.).

06. `acceleration`: Time to accelerate from 0 to 60 mph (sec.).

07. `year`: Model year (modulo 100).

08. `origin`: Origin of car (1. American, 2. European, 3. Japanese).

09. `name`: Vehicle name.

The orginal data contained 408 observations but 16 observations with missing values were removed.

#### Source
This dataset was taken from the StatLib library which is maintained at Carnegie Mellon University. The dataset was used in the 1983 American Statistical Association Exposition.

In [ ]:
#install.packages("ISLR",dependencies=TRUE,repos="https://cloud.r-project.org")
require(ISLR)

In [ ]:
data(Auto)
str(Auto)

In [ ]:
# Discard the 'name' variable, irrelevant for prediction
Auto<-Auto[,-9]
str(Auto)

### Ba.1 Hold-out

### Ba.2 $k$-fold crossvalidation

### Ba.3 Leave-one-out

### Ba.4 Bootstrap

## C. `SAheart` dataset

From the `ElemStatLearn` package, `SAheart` is a data frame with 462 observations on the following 10 variables.

01. `sbp`: systolic blood pressure.

02. `tobacco`: cumulative tobacco (kg).

03. `ldl`: low density lipoprotein cholesterol.

04. `adiposity`: a numeric vector.

05. `famhist`: family history of heart disease, a factor with levels `Absent`, `Present`.

06. `typea`: type-A behavior.

07. `obesity`: a numeric vector.

08. `alcohol`: current alcohol consumption.

09. `age`: age at onset

10. `chd`: response, coronary heart disease

##### Details

A retrospective sample of males in a heart-disease high-risk region of the Western Cape, South Africa. There are roughly two controls per case of CHD. Many of the CHD positive men have undergone blood pressure reduction treatment and other programs to reduce their risk factors after their CHD event. In some cases the measurements were made after these treatments. These data are taken from a larger dataset, described in Rousseauw et al, 1983, South African Medical Journal.

In [ ]:
#install.packages("ElemStatLearn",dependencies=TRUE,repos="https://cloud.r-project.org")
require(ElemStatLearn)
data(SAheart)

In [ ]:
n<-nrow(SAheart)
ntrain<-ceiling(0.60*n)
Itrain<-sample(1:n,ntrain,replace=FALSE)
n<-nrow(SAheart)
ntrain<-ceiling(0.60*n)
Itrain<-sample(1:n,ntrain,replace=FALSE)
SAheart.train<-SAheart[Itrain,]
SAheart.test<-SAheart[-Itrain,]

## Ca. SAheart data with logistic regression

In [ ]:
SAheart.logit1<-glm(chd~.,data=SAheart.train,family=binomial)
summary(SAheart.logit1)

Prediction of probabilities that the binary 0/1 response takes the value 1 (here `chd=1`):

In [ ]:
SAheart.pred<-predict(SAheart.logit1,newdata=SAheart.test,type="response")
str(SAheart.pred)

A _crisp_ prediction of 0 or 1 is obtained from the above by taking a cut point, e.g., $L=0.5$ (this is not a mandatory value, it may depend on the problem or on the _a priori_ probabilites) and assign to 0 or 1 according to whether the probability is smaller or larger than this threshold:

In [ ]:
SAheart.pred.crisp<-1*(SAheart.pred>=0.5)
C<-table("True"=SAheart.test$chd,"Predicted"=SAheart.pred.crisp)
C

### Ca.1 Hold-out

### Ca.2 $k$-fold crossvalidation

### Ca.3 Leave-one-out

### Ca.4 Bootstrap

## Cb. `SAheart` with Fisher's linear discriminant analysis

In [ ]:
require(MASS)
SAheart.lda1<-lda(chd~.,data=SAheart.train)
SAheart.pred<-predict(SAheart.lda1,newdata=SAheart.test)
C<-table("True"=SAheart.test$chd,"Predicted"=SAheart.pred$class)
C

### Cb.1 Hold-out

### Cb.2 $k$-fold crossvalidation

### Cb.3 Leave-one-out

### Cb.4 Bootstrap

## Cb. `SAheart` with Quadratic discriminant

In [ ]:
SAheart.qda1<-qda(chd~.,data=SAheart.train)
SAheart.pred<-predict(SAheart.qda1,newdata=SAheart.test)
C<-table("True"=SAheart.test$chd,"Predicted"=SAheart.pred$class)
C

# D. `Default` data set

From the `ISLR` package. A simulated data set containing information on ten thousand customers. The aim here is to predict which customers will default on their credit card debt.

A data frame with 10000 observations on the following 4 variables.

01. `default`: A factor with levels `No` and `Yes` indicating whether the customer defaulted on their debt

02. `student`: A factor with levels `No` and `Yes` indicating whether the customer is a student

03. `balance`: The average balance that the customer has remaining on their credit card after making their monthly payment

04. `income`: Income of customer

In [ ]:
#install.packages("ISLR",dependencies=TRUE,repos="https://cloud.r-project.org")
require(ISLR)
data(Default)
str(Default)

## Da. `Default` data set with Fisher's linear discriminant

In [ ]:
n<-nrow(Default)
ntrain<-ceiling(0.6*n)
set.seed(24025)         # An arbitrary value, fixed for the sake of reproducibility of results
Itrain<-sample(1:n,ntrain,replace=FALSE)
Default.train<-Default[Itrain,]
Default.test<-Default[-Itrain,]

In [ ]:
require(MASS)
Default.lda<-lda(default~.,data=Default.train)
Default.pred<-predict(Default.lda,newdata=Default.test)
C<-table("True"=Default.test$default,"Predicted"=Default.pred$class)
C

## `Smarket` data set

This is an S&P Stock Market Data set. Daily percentage returns for the S&P 500 stock index between 2001 and 2005.
Contained in the `ISLR` package as a `data.frame` with 1250 observations on the following 9 variables.

01. `Year`: The year that the observation was recorded

02. `Lag1`: Percentage return for previous day

03. `Lag2`: Percentage return for 2 days previous

04. `Lag3`: Percentage return for 3 days previous

05. `Lag4`: Percentage return for 4 days previous

06. `Lag5`: Percentage return for 5 days previous

07. `Volume`: Volume of shares traded (number of daily shares traded in billions)

08. `Today`: Percentage return for today

09. `Direction`: A factor with levels `Down` and `Up` indicating whether the market had a positive or negative return on a given day.

In [ ]:
require(ISLR)
data(Smarket)
str(Smarket)

# Appendix: dedicated R packages

Many R packages contain functions implementing methods for validating prediction procedures, for instance:
- `caret`, 
- `CVST`, 
- `cvTools`, 
- `dprep`, 
- `sortinghat`. 

One of them (`CVST`) is devoted to a validation technique more sophisticated than the four in this notebook, and the rest are generic, with application to some prediction methods than those in our course. 